In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('2021_uniswap_wbtc_usdc_swaps_corrected.json')
df.head()

,timestamp,datetime,blockNumber,txHash,sender,recipient,amount0,amount1,tick,executionPrice
0,2021-05-05 21:42:13,2021-05-05 21:42:13+00:00,12376730,0x9bf327df628323d51256f64e63e26acc15559776f646...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xe5C27478cDDF77670b16352D6B5752F60a96c71f,0.004221,-240.633362,63490,57004.563049
1,2021-05-05 21:51:06,2021-05-05 21:51:06+00:00,12376776,0xd751b357b27be36d0b98faeaf9e1d078117ff3215f79...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xE3c14271E10b41fdF603c845434493884a8dB4d9,0.007017,-400.000000,63490,57003.524243
2,2021-05-05 21:57:19,2021-05-05 21:57:19+00:00,12376802,0xccbe191bb29b7c89d941bd3f4de81aeb8a57c9e253a0...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0x00dbc6E6b3420615f595FCB9DCD0D4a0e4514201,0.087727,-5000.000000,63487,56995.268709
3,2021-05-05 21:59:00,2021-05-05 21:59:00+00:00,12376811,0x518fefce14365f42f5144da5503700e285540eade990...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xe623031eC40DAA3EF5f87b408a95c9221035291f,0.060222,-3431.560270,63485,56982.188460
4,2021-05-05 22:08:57,2021-05-05 22:08:57+00:00,12376869,0xefd3b31c172ff21b8987ba4b224110f9e22dc3d9c726...,0xE592427A0AEce92De3Edee1F18E0157C05861564,0xE592427A0AEce92De3Edee1F18E0157C05861564,0.030485,-1736.872329,63484,56974.168139


In [3]:
df.shape

(3801, 10)

In [4]:
df.columns

Index(['timestamp', 'datetime', 'blockNumber', 'txHash', 'sender', 'recipient',
       'amount0', 'amount1', 'tick', 'executionPrice'],
      dtype='object')

In [5]:
df = df.drop(columns=['txHash', 'sender', 'recipient'])
df.head(5)

,timestamp,datetime,blockNumber,amount0,amount1,tick,executionPrice
0,2021-05-05 21:42:13,2021-05-05 21:42:13+00:00,12376730,0.004221,-240.633362,63490,57004.563049
1,2021-05-05 21:51:06,2021-05-05 21:51:06+00:00,12376776,0.007017,-400.000000,63490,57003.524243
2,2021-05-05 21:57:19,2021-05-05 21:57:19+00:00,12376802,0.087727,-5000.000000,63487,56995.268709
3,2021-05-05 21:59:00,2021-05-05 21:59:00+00:00,12376811,0.060222,-3431.560270,63485,56982.188460
4,2021-05-05 22:08:57,2021-05-05 22:08:57+00:00,12376869,0.030485,-1736.872329,63484,56974.168139


In [6]:
# # Group by 1-minute bins
# ohlcv = df.resample('1min', on='datetime').agg({
#     'executionPrice': ['first', 'max', 'min', 'last'],
#     'amount0': lambda x: x.abs().sum()  # volume in token1 terms
# })

# ohlcv.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
# ohlcv = ohlcv.dropna().reset_index()

# ohlcv.head(5)

# # export into CSV
# ohlcv.to_csv('uniswap_wbtc_usdc_ohlcv.csv', index=False)

In [7]:
def create_ohlcv(df, timeframe, filename):
    ohlcv = df.resample(timeframe, on='datetime').agg({
        'executionPrice': ['first', 'max', 'min', 'last'],
        'amount0': lambda x: x.abs().sum()
    })
    ohlcv.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    ohlcv = ohlcv.dropna().reset_index()
    ohlcv.to_csv(filename, index=False)
    return ohlcv

In [8]:
# Then use it like:
timeframes = ['5min', '15min', '30min', '1h', '4h', '6h', '12h', '1d', '1w']
for tf in timeframes:
    create_ohlcv(df, tf, f'data/{tf}_uniswap_wbtc_usdc_ohlcv.csv')

/tmp/ipykernel_113402/3993398381.py:2: FutureWarning: 'w' is deprecated and will be removed in a future version, please use 'W' instead.
  ohlcv = df.resample(timeframe, on='datetime').agg({
